In [ ]:
%pip install wordcloud
%pip install seaborn
%pip install ipywidgets
%pip install nltk

In [ ]:
from collections import Counter
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from wordcloud import WordCloud, STOPWORDS
sns.set_theme()
warnings.filterwarnings("ignore")
import pandas as pd
import re

## ETL

In [ ]:
#Extraction
projects = pd.read_csv('projects.csv')
tags = pd.read_csv('tags.csv')
projects.head()

In [ ]:
tags.head()

In [ ]:
#Transform
df = pd.merge(tags,projects,on='id')
df.head()

In [ ]:
df.tag.isnull()

In [ ]:
df = df.loc[df.tag.notnull(),:]

In [ ]:
#Save the transformed data
df.to_csv('labeled_projects.csv',index=False)

## EDA

In [ ]:
df = pd.read_csv('labeled_projects.csv')

In [ ]:
df.head()

In [ ]:
#Tags distribution
df.tag.hist()

In [ ]:
#Wordcloud of description feature
def get_wc(df,tag):
    tag_titles = df.loc[df.tag==tag,'title']
    cloud = WordCloud(stopwords=STOPWORDS,collocations=False,width=500, height=300).generate(' '.join(tag_titles.to_list()))
    plt.imshow(cloud)


In [ ]:
get_wc(df,'natural-language-processing')

In [ ]:
get_wc(df,'mlops')

In [ ]:
get_wc(df,'time-series')

In [ ]:
df.tag.unique()

## Preprocessing

In [105]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')
# nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/momalijaz/nltk_data...


True

In [ ]:
df.head()

In [ ]:
df['text'] = df.title +' ' + df.description
df.text

In [72]:
# Clean the text
def clean_text(text,stopwords):

    text = text.lower()
    
    #Remove non-alphanumeric
    text = re.sub(r"([!\"'#$%&()*\+,-./:;<=>?@\\\[\]^_`{|}~])", r" \1 ", text )  
    text = re.sub("[^A-Za-z0-9]+", " ", text) 

    #Remove links
    text = re.sub('http\S+',"",text)
    
    #tokenize
    text = text.split()

    #Remove  stopwords 
    new_text = []
    for word in text:   
        if word not in stopwords:
            new_text.append(word)
    text = new_text 
    
    # Stem
    stemmer = PorterStemmer()
    text = stemmer.stem(' '.join(text))
    
    return text

In [85]:
# 2. lower all the text tokens
df['text'].apply(clean_text,args=(STOPWORDS,))[543]

['object',
 'goal',
 'navigation',
 'using',
 'goal',
 'oriented',
 'semantic',
 'exploration',
 'embodied',
 'interactive',
 'learning',
 'object',
 'detection',
 'using',
 'semantic',
 'curiosity',
 'learn',
 'exploration',
 'policy',
 'set',
 'training',
 'environments']

In [100]:
stemmer = PorterStemmer()
stemmer.stem('i am a hard working person')

'i am a hard working person'

In [107]:
lematizer = WordNetLemmatizer()
lematizer.lemmatize('pots')


'pot'